In [1]:
import sys
import os
# Add the parent directory to the Python path

# Change the current working directory to /app
os.chdir('/app')
from dotenv import load_dotenv
load_dotenv(dotenv_path='config/.env')
database_path = os.getenv('DATABASE_PATH')
from utils.db.fetch import fetch_entries
print(f'DATABASE_PATH: {database_path}')
market_name = 'indian_equity'
timeframe='1d'

DATABASE_PATH: database/db/ohlcv_data.db


In [36]:
import cloudscraper
import time

def fetch_sol_addresses_with_cloudscraper():
    url = "https://app.geckoterminal.com/api/p1/solana/pools"
    params = {
        "include": "dex,dex.network,dex.network.network_metric,tokens",
        "page": 1,
        "include_network_metrics": "true",
        "include_meta": "1",
        "volume_24h[gte]": "1000000",
        "pool_creation_hours_ago[lte]": "1200",
        "has_social": "1",
        "sort": "-24h_volume",
        "networks": "solana",
    }
    headers = {
        "User-Agent": "PostmanRuntime/7.42.0",
        "Accept": "*/*",
        "Accept-Encoding": "gzip, deflate, br",
        "Connection": "keep-alive",
    }

    scraper = cloudscraper.create_scraper()  # Create a scraper object that bypasses Cloudflare
    scraper.headers.update(headers)  # Update headers for all requests
    
    sol_addresses = []

    while True:
        response = scraper.get(url, params=params)

        if response.status_code == 403:
            print("403 Forbidden: Ensure you are authorized to access the API.")
            break
        elif response.status_code != 200:
            print(f"Error: Unable to fetch data, status code {response.status_code}")
            break

        print(response.text)
        data = response.json()
        print(data)
        pools = data.get("data", [])
        
        for pool in pools:
            attributes = pool.get("attributes", {})
            name = attributes.get("name", "")
            address = attributes.get("address", "")
            if name.lower().endswith("/ sol"):
                sol_addresses.append(address)

        links = data.get("links", {})
        next_page = links.get("next")
        if not next_page:
            break

        params["page"] += 1
        time.sleep(3)

    return sol_addresses



In [37]:

addresses = fetch_sol_addresses_with_cloudscraper()
print(f"Found {len(addresses)} addresses ending with '/ SOL':")
print(addresses)

{"data":[{"id":"168691032","type":"pool","attributes":{"address":"93tjgwff5Ac5ThyMi8C4WejVVQq4tuMeMuYW1LEYZ7bu","name":"CHILLGUY / SOL","from_volume_in_usd":"235066417.118091","to_volume_in_usd":"235066417.118091","api_address":"93tjgwff5Ac5ThyMi8C4WejVVQq4tuMeMuYW1LEYZ7bu","swap_count_24h":214064,"price_percent_change":"+755.48%","price_percent_changes":{"last_5m":"+2.92%","last_15m":"-1.8%","last_30m":"-10.11%","last_1h":"+14.79%","last_6h":"+22.89%","last_24h":"+756.33%"},"pool_fee":null,"base_token_id":"36213287","token_value_data":{"4045901":{"fdv_in_usd":140536198957.67166,"market_cap_in_usd":0.0,"market_cap_to_holders_ratio":175059.53541147636},"36213287":{"fdv_in_usd":250330912.67063227,"market_cap_in_usd":250330913.73165035,"market_cap_to_holders_ratio":4320.508940536517}},"price_in_usd":"0.250350929824427","reserve_in_usd":"3386085.45597","pool_created_at":"2024-11-15T20:08:23.000Z","aggregated_network_metrics":{"total_swap_volume_usd_24h":"17674352897.2117558620488373","tota

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [32]:
import cloudscraper

url = "https://app.geckoterminal.com/api/p1/solana/pools?include=dex%2Cdex.network%2Cdex.network.network_metric%2Ctokens&page=4&include_network_metrics=true&include_meta=1&volume_24h%5Bgte%5D=1000000&pool_creation_hours_ago%5Blte%5D=1200&has_social=1&sort=-24h_volume&networks=solana"

headers = {
    "User-Agent": "PostmanRuntime/7.42.0",
    "Accept": "*/*",
    "Accept-Encoding": "gzip, deflate, br",
    "Connection": "keep-alive",
}

scraper = cloudscraper.create_scraper()  # Handles Cloudflare and Brotli encoding
response = scraper.get(url, headers=headers)

if response.status_code == 200:
    print(response.text)  # Decoded response content
else:
    print(f"Failed with status code: {response.status_code}")


{"data":[{"id":"168719510","type":"pool","attributes":{"address":"DiRTSpizmJfqNKUsCiQPxacwppYV8BvURVERTQifQUgA","name":"scihub / SOL","from_volume_in_usd":"1806510.880466","to_volume_in_usd":"1806510.880466","api_address":"DiRTSpizmJfqNKUsCiQPxacwppYV8BvURVERTQifQUgA","swap_count_24h":2683,"price_percent_change":"+51.98%","price_percent_changes":{"last_5m":"+2.44%","last_15m":"+7.15%","last_30m":"+11.43%","last_1h":"+13.37%","last_6h":"+132.84%","last_24h":"+51.98%"},"pool_fee":null,"base_token_id":"36221254","token_value_data":{"4045901":{"fdv_in_usd":140271291784.16623,"market_cap_in_usd":0.0,"market_cap_to_holders_ratio":174750.20653418757},"36221254":{"fdv_in_usd":40562601.22213444,"market_cap_in_usd":0.0,"market_cap_to_holders_ratio":2739.5011971888252}},"price_in_usd":"0.0408993202492667","reserve_in_usd":"303676.92122","pool_created_at":"2024-11-16T13:28:25.000Z","aggregated_network_metrics":{"total_swap_volume_usd_24h":"17712210376.1868450757900473","total_swap_volume_usd_48h_2